In [218]:
import dash
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from dash import html,dcc,Dash,dash_table
from dash.dependencies import Input, Output
app = Dash()
colors_no_show = ["#ff00ff", "#00ffff", "#ffff00", "#ff1493", "#00ff00"]
colors_age_gender = ["#ff4500", "#00ced1", "#ff69b4", "#7fff00", "#ffd700"]
colors_day_of_week = ["#9400d3", "#ff6347", "#00fa9a", "#ffdab9", "#1e90ff"]
colors_neighborhood = ["#ff00ff", "#00ffcc", "#ffcc00", "#ff3366", "#00ff99"]
color_disease = ["#ff0066", "#33ffcc", "#ffff66", "#ff9933", "#66ff33"]
colors_delay_appointments = ["#ff33ff", "#00ff66", "#66ccff", "#ffcc66", "#ff3300"]
template_style = 'plotly_dark'
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Preprocessing**

In [219]:
df = pd.read_csv(r"C:\KaggleV2-May-2016.csv")

In [220]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [221]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [222]:
df[df.duplicated()]

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show


In [223]:
df.isnull().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [224]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'], dayfirst=True)
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], dayfirst=True)
df['DelayDays'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

C:\Users\it\AppData\Local\Temp\ipykernel_976\2525798971.py:1: UserWarning:

Parsing dates in %Y-%m-%dT%H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.

C:\Users\it\AppData\Local\Temp\ipykernel_976\2525798971.py:2: UserWarning:

Parsing dates in %Y-%m-%dT%H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.



In [225]:
df[['ScheduledDay', 'AppointmentDay']].head()

,ScheduledDay,AppointmentDay
0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00
1,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00
2,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00
3,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00
4,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00


In [226]:
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.strftime('%d-%m-%Y')

In [227]:
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], dayfirst = True)
df['day'] = df['AppointmentDay'].dt.day_name()

In [228]:
df[['AppointmentDay','ScheduledDay']].head()

,AppointmentDay,ScheduledDay
0,2016-04-29 00:00:00+00:00,29-04-2016
1,2016-04-29 00:00:00+00:00,29-04-2016
2,2016-04-29 00:00:00+00:00,29-04-2016
3,2016-04-29 00:00:00+00:00,29-04-2016
4,2016-04-29 00:00:00+00:00,29-04-2016


In [229]:
df = df.rename(columns={"Handcap": "Handicap"})

In [230]:
df['No-show'] = df['No-show'].map({'Yes' : 'No' , 'No' : 'Yes' })
df.rename(columns={"No-show" : "Attendance"},inplace = True)

In [231]:
df['Gender'] = df['Gender'].map({'F' : 'Female' , 'M' : 'Male' })

In [232]:
df.drop(columns = ["PatientId"],inplace = True)

In [233]:
df.head()

,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handicap,SMS_received,Attendance,DelayDays,day
0,5642903,Female,29-04-2016,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,Yes,-1,Friday
1,5642503,Male,29-04-2016,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,Yes,-1,Friday
2,5642549,Female,29-04-2016,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,Yes,-1,Friday
3,5642828,Female,29-04-2016,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,Yes,-1,Friday
4,5642494,Female,29-04-2016,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,Yes,-1,Friday


In [234]:
df =df[df['Age'] >= 1 ]
df =df[df['Age'] <= 100 ]
df.Age.value_counts()

Age
1      2273
52     1746
49     1652
53     1651
56     1635
       ... 
96       17
97       11
98        6
100       4
99        1
Name: count, Length: 100, dtype: int64

## Visualization

#### Attendance Vs Dis Att

In [235]:
yes_att = df[df['Attendance'] == 'Yes'].shape[0]
no_att  = df[df['Attendance'] == 'No' ].shape[0]

age_attendance = df.groupby(['Age', 'Attendance']).size().reset_index(name='Count')

In [236]:
attendance_df = pd.DataFrame({
    'Attendance': ['Yes', 'No'],
    'Count': [yes_att, no_att]
})

In [237]:
fig_attendance_bar = px.bar(attendance_df,
             x='Attendance',
             y='Count' ,
             color='Attendance',
             color_discrete_sequence=colors_no_show,
             template=template_style,
)
fig_attendance_bar.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_no_show[0]),
    title_x=0.5
)

#### Atten Vs Gender

In [238]:
'''
1.Sum of gender and attendance , then put them in new col
2.total present and absent in one col
3.percentage of present and absent in one col
4.count male and female
5.index to Gender , gender to count
'''
gender_attendance = df.groupby(['Gender','Attendance']).size().reset_index(name = 'count of gender')
total_by_gender = gender_attendance.groupby('Gender')['count of gender'].transform('sum')
gender_attendance['percentage'] = (gender_attendance['count of gender'] / total_by_gender) * 100


male_vs_female = df['Gender'].value_counts().reset_index()
male_vs_female.columns = ['Gender','Count']
gender_distribution = male_vs_female.copy()
genders_list = ['Male','Female','All']
pie_charts = []

In [239]:
for gender in genders_list:
    if gender == 'All':
        data = df.groupby('Attendance').size().reset_index(name='Count')
    else:
        data = df[df['Gender'] == gender].groupby('Attendance').size().reset_index(name='Count')
    total = data['Count'].sum()
    if total == 0:
        data['percentage'] = 0
    else:
        data['percentage'] = (data['Count'] / total) * 100
    fig = px.pie(
        data,
        names='Attendance',
        values='percentage',
        title=f'Attendance Distribution for {gender}',
        color_discrete_sequence=colors_age_gender,
        template=template_style
    )
    pie_charts.append(dcc.Graph(figure=fig, id=f'pie-{gender}'))

In [240]:
fig_gender_overall = px.pie(
    gender_distribution,
    names='Gender',
    values='Count',
    title='Overall Gender Distribution',
    color_discrete_sequence=colors_age_gender,
    template=template_style
)
fig_gender_overall.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_age_gender[1]),
    title_x=0.5
)
fig_gender_overall.update_layout(
    width=500,   # العرض بالبكسل
    height=400   # الارتفاع بالبكسل
)

In [241]:
gender_table = gender_attendance.pivot(index='Gender', columns='Attendance', values='count of gender').fillna(0)
gender_table['Total'] = gender_table.sum(axis=1)

#### attendance Vs Age

In [242]:
age =age_attendance.groupby('Age').size().reset_index(name = 'Attendance')
age

,Age,Attendance
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
...,...,...
95,96,2
96,97,2
97,98,2
98,99,1


In [243]:
fig_age = px.bar(age_attendance,
             x='Age',
             y='Count',
             color='Attendance',
             barmode='group',
             title='Attendance vs Age ',
             color_discrete_sequence=colors_age_gender,
             template=template_style
)
fig_age.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_age_gender[2]),
    title_x=0.5
)

#### Attendance Per Disease

In [244]:
attendance_per_disease = df.groupby(
    ['Hipertension', 'Diabetes', 'Alcoholism', 'Handicap', 'Attendance']
).size().reset_index(name='Count')
attendance_per_disease

,Hipertension,Diabetes,Alcoholism,Handicap,Attendance,Count
0,0,0,0,0,No,16964
1,0,0,0,0,Yes,63610
2,0,0,0,1,No,216
3,0,0,0,1,Yes,867
4,0,0,0,2,No,13
5,0,0,0,2,Yes,84
6,0,0,0,3,No,2
7,0,0,0,3,Yes,7
8,0,0,0,4,No,1
9,0,0,0,4,Yes,2


In [245]:
df_clean = attendance_per_disease.copy()

df_clean['Hipertension'] = df_clean['Hipertension'].map({0: 'No Hypertension', 1: 'Has Hypertension'})
df_clean['Diabetes'] = df_clean['Diabetes'].map({0: 'No Diabetes', 1: 'Has Diabetes'})
df_clean['Alcoholism'] = df_clean['Alcoholism'].map({0: 'No Alcoholism', 1: 'Has Alcoholism'})
df_clean['Handicap'] = df_clean['Handicap'].map({
    0: 'No Handicap',
    1: 'Mild Handicap',
    2: 'Moderate Handicap',
    3: 'Severe Handicap',
    4: 'Very Severe Handicap'
})

In [246]:
fig_disease= px.sunburst(df_clean,
                  path=['Attendance', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handicap'],
                  values='Count',
                  title='Attendance vs No-show by Disease and Handicap',
                  color='Count',
                  color_continuous_scale='Blues',
                  color_discrete_sequence=color_disease,
                  template=template_style
)
fig_disease.update_layout(
    height=450,
    title_font=dict(size=20, color='#f15bb5'),
    title_x=0.5
)

#### Attendance Per Scholarship

In [247]:
scholar_attendance = df.groupby(['Scholarship', 'Attendance']).size().reset_index(name='Count')
scholar_attendance

,Scholarship,Attendance,Count
0,0,No,19106
1,0,Yes,77065
2,1,No,2571
3,1,Yes,8238


In [248]:
fig_scholar = px.bar(scholar_attendance,
             x='Scholarship',
             y='Count',
             color='Attendance',
             barmode='group',
             title='Attendance vs Scholarship (with No-show)',
             color_discrete_sequence=colors_no_show,
             template=template_style
)
fig_scholar.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_no_show[1]),
    title_x=0.5
)

#### Attendance vs date

In [249]:
day_counts = df.day.value_counts().reset_index()
day_counts.columns = ['day', 'Count']

In [250]:
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['day'] = pd.Categorical(df['day'], categories=ordered_days, ordered=True)
attendance_by_day = df.groupby(['day', 'Attendance'], observed=True).size().reset_index(name='Count')

In [251]:
fig_day = px.bar(
    attendance_by_day,
    x='day',
    y='Count',
    color='Attendance',
    barmode='group',
    title='Attendance vs Absence by Day of Week',
    color_discrete_sequence=colors_day_of_week,
    template=template_style
)
fig_day.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_day_of_week[1]),
    title_x=0.5
)

#### Effect of Delay Between Scheduling and Appointment on Attendance

In [252]:
fig_dealy = px.histogram(
    df,
    x='DelayDays',
    color='Attendance',
    nbins=30,
    barmode='overlay',
    title='Effect of Delay Between Scheduling and Appointment on Attendance',
    color_discrete_sequence=colors_delay_appointments,
    template=template_style
)
fig_dealy.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_delay_appointments[0]),
    title_x=0.5
)


#### SMS

In [253]:
sms_effect = df.groupby(['SMS_received', 'Attendance'], observed=True).size().reset_index(name='Count')


In [254]:
fig_sms = px.line(
    sms_effect,
    x='SMS_received',
    y='Count',
    color='Attendance',
    title='Attendance vs SMS Delay',
    markers=True,
    color_discrete_map={'Yes': 'green', 'No': 'red'},
    color_discrete_sequence=color_disease,
    template=template_style
)
fig_sms.update_layout(
    height=450,
    title_font=dict(size=20, color=color_disease[0]),
    title_x=0.5
)

In [255]:
neighborhood_attendance = df.groupby(['Neighbourhood', 'Attendance'], observed=True).size().reset_index(name='Count')

In [256]:
fig_neighborhood = px.bar(
    neighborhood_attendance,
    x='Neighbourhood',
    y='Count',
    color='Attendance',
    barmode='group',
    title='Attendance by Neighborhood',
    color_discrete_map={'Yes': 'green', 'No': 'red'},
    color_discrete_sequence=colors_neighborhood,
    template=template_style
)
fig_neighborhood.update_layout(
    height=450,
    title_font=dict(size=20, color=colors_neighborhood[0]),
    title_x=0.5
)


In [259]:
from dash import Dash, html, dcc, dash_table, Output, Input
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([

    html.H1("Medical Appointment Attendance Dashboard", style={'textAlign': 'center'}),

    # attendance bar
    html.Div(dcc.Graph(figure=fig_attendance_bar), style={'margin-bottom': '20px'}),

    # overall gender pie
    html.Div(dcc.Graph(figure=fig_gender_overall), style={'margin-bottom': '20px'}),

    html.H2("Attendance by Gender", style={'marginTop': '20px'}),
    html.Div(pie_charts, style={'display': 'block'}),  # كل Pie تحت بعض

    html.H3("Gender Summary Table"),
    dash_table.DataTable(
        data=gender_table.reset_index().to_dict('records'),
        columns=[{'name': c, 'id': c} for c in gender_table.reset_index().columns],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '6px'}
    ),

    # Attendance by Age (Top N)
    html.H3("Attendance by Age (Top Age)"),
    html.Div([
        dcc.Dropdown(
            id='top-n-age-dropdown',
            options=[{'label': str(i), 'value': i} for i in [5, 10, 20, 50]],
            value=10,
            clearable=False,
            style={'width': '200px'}
        )
    ], style={'margin-bottom': '20px'}),
    html.Div(dcc.Graph(id='age-graph'), style={'margin-top': '20px'}),

    # Other charts
    html.Div(dcc.Graph(figure=fig_disease), style={'margin-top': '20px'}),
    html.Div(dcc.Graph(figure=fig_scholar), style={'margin-top': '20px'}),
    html.Div(dcc.Graph(figure=fig_day), style={'margin-top': '20px'}),
    html.Div(dcc.Graph(figure=fig_dealy), style={'margin-top': '20px'}),
    html.Div(dcc.Graph(figure=fig_sms), style={'margin-top': '20px'}),

    # Attendance by Neighborhood (Top N)
    html.H3("Attendance by Neighborhood (Top N)"),
    html.Div([
        dcc.Dropdown(
            id='top-n-dropdown',
            options=[{'label': str(i), 'value': i} for i in [5, 10, 20, 50]],
            value=10,
            clearable=False,
            style={'width': '200px'}
        ),
        dcc.Graph(id='neighborhood-graph')
    ], style={'marginTop': '10px'}),

], style={'maxWidth': '1200px', 'margin': '0 auto', 'padding': '20px'})


# Callback for Neighborhood graph
@app.callback(
    Output('neighborhood-graph', 'figure'),
    Input('top-n-dropdown', 'value')
)
def update_neighborhood(top_n):
    top_neigh = neighborhood_attendance.groupby('Neighbourhood')['Count'].sum().nlargest(top_n).index
    filtered = neighborhood_attendance[neighborhood_attendance['Neighbourhood'].isin(top_neigh)]
    fig = px.bar(filtered, x='Neighbourhood', y='Count', color='Attendance', barmode='group',
                 title=f'Attendance by Neighborhood (Top {top_n})')
    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    return fig


# Callback for Age graph
@app.callback(
    Output('age-graph', 'figure'),
    Input('top-n-age-dropdown', 'value')
)
def update_age_graph(top_n):
    top_age = age_attendance.groupby('Age')['Count'].sum().nlargest(top_n).index
    filtered = age_attendance[age_attendance['Age'].isin(top_age)]
    fig = px.bar(
        filtered,
        x='Age',
        y='Count',
        color='Attendance',
        barmode='group',
        title=f'Attendance by Age (Top {top_n})',
        color_discrete_sequence=colors_age_gender,
        template=template_style
    )
    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    return fig


if __name__ == '__main__':
    app.run(debug=True)
